In [1]:
import numpy as np
import trimesh
import tqdm 

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    # 1. compute area of each triangles
    area = np.zeros(faces.shape[0])
    v0 = vertices[faces[:, 0]]
    v1 = vertices[faces[:, 1]]
    v2 = vertices[faces[:, 2]]
    cross = np.cross(v1 - v0, v2 - v0)
    area = np.linalg.norm(cross, axis=1) / 2
    prob = area / np.sum(area)
    sampled_face_indices = np.random.choice(
        len(faces), 
        size=sample_num,
        p=prob,
        replace=True
    )
    u = np.random.rand(sample_num)
    v = np.random.rand(sample_num)
    mask = u + v > 1  # 处理超出三角形区域的情况
    u[mask] = 1 - u[mask]
    v[mask] = 1 - v[mask]
    w = 1 - u - v  # 第三个重心坐标
    
    # 获取对应面片的顶点（向量化操作）
    sampled_v0 = vertices[faces[sampled_face_indices, 0]]
    sampled_v1 = vertices[faces[sampled_face_indices, 1]]
    sampled_v2 = vertices[faces[sampled_face_indices, 2]]
    uniform_pc = (w[:, None] * sampled_v0 +
        u[:, None] * sampled_v1 +
        v[:, None] * sampled_v2)
    return area, prob, uniform_pc
        

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------
    results = np.zeros((sample_num, pc.shape[1]))
    results[0] = pc[np.random.choice(pc.shape[0])]
    distances = np.full(pc.shape[0], np.inf)
    for i in range(1, sample_num):
        for j in range(pc.shape[0]):
            dist = np.linalg.norm(pc[j] - results[:i], axis=1)
            distances[j] = np.min(dist)
        idx = np.argmax(distances)
        results[i] = pc[idx]

    return results

In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [6]:
# task 3: metrics

from earthmover.earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------

CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0
CD = []
EMD = []

for i in tqdm.tqdm(range(5)):
    # uniform sampling
    _,_, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, final_sample_num)
    
    # FPS
    _,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

    # compute metrics
    cd = np.mean(np.min(np.linalg.norm(uniform_pc[:, None] - fps_pc[None], axis=2), axis=1)) + \
         np.mean(np.min(np.linalg.norm(fps_pc[:, None] - uniform_pc[None], axis=2), axis=1))
    CD.append(cd)
    emd = earthmover_distance(uniform_pc, [tuple(point) for point in fps_pc])
    EMD.append(emd)

CD_mean = np.mean(CD)
CD_var = np.var(CD)
EMD_mean = np.mean(EMD)
EMD_var = np.var(EMD)

print('CD mean: ', CD_mean)
print('CD var: ', CD_var)
print('EMD mean: ', EMD_mean)
print('EMD var: ', EMD_var)
# Visualization. For you to check your code
# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\zlib1.dll...
load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\abseil_dll.dll...
load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\utf8_validity.dll...
load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\re2.dll...
load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\libprotobuf.dll...
load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\highs.dll...
load d:\anaconda\envs\cvlab3\lib\site-packages\ortools\.libs\ortools.dll...


  0%|          | 0/5 [00:00<?, ?it/s]

move 0.001953125 dirt from [34.8502402  13.34889452 33.48180444] to (35.92117493190026, 12.794851109344279, 32.130127609837835) for a cost of 0.0035377426745145605
move 0.001953125 dirt from [38.71619277 13.03897532 28.35902519] to (38.299926330405256, 13.990793305779583, 26.194434839033082) for a cost of 0.0046894066044487846
move 0.001953125 dirt from [31.36729104 15.0531359  22.22995135] to (32.66736072327141, 17.022692481765976, 21.772199600784816) for a cost of 0.0046951723491237465
move 0.001953125 dirt from [12.23123424 33.22743759 22.93265724] to (9.81511449886161, 33.32147183448666, 23.625168418527288) for a cost of 0.004912429240488445
move 0.001953125 dirt from [20.77364281 49.59524992 35.19962097] to (19.484561266504244, 50.91557870354236, 36.29603771705244) for a cost of 0.004192231202611265
move 0.001953125 dirt from [52.26699459 16.78371779 37.68838043] to (51.51663101672205, 21.023914673001265, 38.9040658171637) for a cost of 0.008739051748320008
move 0.001953125 dirt f

 20%|██        | 1/5 [00:40<02:41, 40.39s/it]

move 0.001953125 dirt from [52.29395566 11.9606398  35.86494424] to (50.81851268029655, 11.495913257801405, 29.20828907603364) for a cost of 0.013347714088049847
move 0.001953125 dirt from [16.4530572  54.27399398 26.534084  ] to (17.866686941205273, 55.06440082845014, 25.85387579057697) for a cost of 0.0034309325701248825
move 0.001953125 dirt from [53.95751528 27.24483904 33.8024827 ] to (53.992564195890814, 25.781042180964963, 34.06431274275972) for a cost of 0.0029051607778081655
move 0.001953125 dirt from [31.28703301 16.88307907 41.82581121] to (34.965094651392874, 16.173656479073127, 40.33620473400809) for a cost of 0.00787338222774491
move 0.001953125 dirt from [22.60289845 10.23864941 25.21640471] to (22.925915682334644, 9.377295010116203, 25.12664898342666) for a cost of 0.0018052704106958665
move 0.001953125 dirt from [18.6458293  51.54264468 30.59550595] to (17.49926456340089, 51.697893695337314, 31.312321675340563) for a cost of 0.002658358636600029
move 0.001953125 dirt f

...

move 0.001953125 dirt from [24.91063515 13.53391033 34.14140157] to (25.635667883441318, 16.253590872616613, 31.98750320441282) for a cost of 0.006922337049551505
move 0.001953125 dirt from [17.5991714  53.16415252 24.69914853] to (16.72613074605975, 52.99462131904684, 24.869818476119935) for a cost of 0.0017687043619140311
move 0.001953125 dirt from [51.0566389  30.22728533 26.93415355] to (50.13424900469455, 32.34389935510613, 29.124320638479418) for a cost of 0.0062156311838028
move 0.001953125 dirt from [16.60077398 32.53004355 25.1215905 ] to (17.02767998792457, 32.186512592355584, 25.9015951876604) for a cost of 0.0018618000221736482
move 0.001953125 dirt from [36.82816545 13.58295096 27.13092855] to (37.589315827121595, 13.259622593644712, 27.690850976461412) for a cost of 0.0019505884521038893
move 0.001953125 dirt from [10.04130349 44.40297142 32.12465312] to (10.057960950156446, 44.39494138235361, 32.600969903952844) for a cost of 0.0009310070355884459
move 0.001953125 dirt f

 80%|████████  | 4/5 [02:35<00:38, 38.65s/it]

move 0.001953125 dirt from [16.24088195 45.94898653 41.38097682] to (17.314048523807507, 45.13935487102239, 41.677451250507076) for a cost of 0.002688714057396035
move 0.001953125 dirt from [46.70576375  9.35338064 22.87930216] to (48.424429332982086, 9.656649003410028, 22.424599970137002) for a cost of 0.0035224202790198745
move 0.001953125 dirt from [25.18326614 15.94825553 29.28313545] to (25.605688398374692, 16.374690010060355, 29.95992136536618) for a cost of 0.0017668238051446084
move 0.001953125 dirt from [24.15995445 44.11541608 36.93605348] to (24.939056473100578, 43.0957380904067, 36.77383555912155) for a cost of 0.0025263034335519154
move 0.001953125 dirt from [44.37331121 14.05215486 34.05370206] to (42.4063470518126, 14.413255394844878, 35.602986495565226) for a cost of 0.004940913653313884
move 0.001953125 dirt from [32.50628172 15.05663244 24.99195174] to (33.98005486555805, 15.531347334188837, 24.881666127404525) for a cost of 0.0030317661954349954
move 0.001953125 dirt

100%|██████████| 5/5 [03:13<00:00, 38.64s/it]

CD mean:  2.7602065861551486
CD var:  0.00021452335249830547
EMD mean:  2.2916508794841985
EMD var:  0.012002454767063391
